# GPT-5 Basic Test Notebook (Azure OpenAI)

This notebook provides examples of the Chat Completions API and the Responses API to quickly validate the basic functionality of GPT-5 in Azure OpenAI. It uses the Azure OpenAI endpoint, key, and deployment name configured in the environment variables.

## Requirements

- Python 3.10+
- `pip install -r GPT-5/requirements.txt`
- `GPT-5/.env` 
  - AZURE_OPENAI_ENDPOINT
  - AZURE_OPENAI_API_KEY
  - AZURE_OPENAI_API_VERSION
  - AZURE_OPENAI_DEPLOYMENT_NAME

## GPT-5 series model comparision 
Reference: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/concepts/models?tabs=global-standard%2Cstandard-chat-completions 

| Model           | Reasoning Support       | Chat Completions API | Responses API | Structured Outputs | Function/Tool Calling     | Context Window (tokens)     | Price per 1M tokens (input/output) |
|-----------------|-------------------------|----------------------|---------------|--------------------|---------------------------|-----------------------------|------------------------------------|
| **GPT-5**       | ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                        | 400,000 (272k in + 128k out) | $1.25 / $10                        |
| **GPT-5-mini**  | ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                       | 400,000 (272k in + 128k out) | $0.25 / $2.00                      |
| **GPT-5-nano**  | ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                       | 400,000 (272k in + 128k out) | $0.05 / $0.40                      |
| **GPT-5-chat**  | 〰️ (undocumented)      | ✅​                  | ✅​           | 〰️ (undocumented)   | 〰️ (undocumented)          | 128,000 (16k max output)     | $1.25 / $10                        |
| **GPT-4.1**     | ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                       | 1,000,000                   | $2.00 / $8.00                      |
| **GPT-4.1-mini**| ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                       | 1,000,000                   | $0.40 / $1.60                      |
| **GPT-4.1-nano**| ✅​                     | ✅​                  | ✅​           | ✅​                | ✅​                       | 1,000,000                   | $0.10 / $0.40                      |
| **GPT-4o**      | ✅​                     | ✅​                  | 〰️ (undocumented) | ✅​           | ✅​                        | 128,000 (+16k out)          | $5.00 / $15.00                     |


> - gpt-5 series models support Reasoning, Chat Completions API, Responses API, Structured Outputs API, and Function/Tool Calling (including parallel calls).
> - As of September 8, 2025, gpt-5-chat (Preview) supports Chat Completions API and Responses API, but does not support Reasoning, Structured Outputs or Function/Tool Calling.


## Chat Completions API Test

In [15]:
import os
import json
import openai
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, EnvironmentCredential
from azure.keyvault.secrets import SecretClient
from io import BytesIO
import gradio as gr
import time

import json
load_dotenv(override=True)

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY", "") if len(os.getenv("AZURE_OPENAI_API_KEY", "")) > 0 else None
azure_openai_gpt5_deployment_name = "gpt-5-mini" # can be "gpt-5" or "gpt-5-chat" or "gpt-5-mini"
azure_openai_gpt41_deployment_name = "gpt-4.1"  # can be "gpt-4.1" or "gpt-4.1-mini" or "gpt-4o"
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "") if len(os.getenv("AZURE_OPENAI_API_VERSION", "")) > 0 else None

try:
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key,
        api_version=aoai_api_version
    )
except (ValueError, TypeError) as e:
    print(e)

# Performance measurement utilities
class PerformanceTimer:
    def __init__(self, test_name):
        self.test_name = test_name
        self.start_time = None
        
    def __enter__(self):
        self.start_time = time.time()
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        end_time = time.time()
        self.execution_time = end_time - self.start_time
        
    def get_time(self):
        return getattr(self, 'execution_time', None)

def measure_and_compare(test_name, gpt5_func, gpt41_func):
    """Generic performance comparison for any test function"""
    print(f"\n=== {test_name} Performance Comparison ===")
    
    results = {}
    
    # Test GPT-5
    print(f"\n--- {azure_openai_gpt5_deployment_name} Results ---")
    try:
        with PerformanceTimer("GPT-5") as timer:
            gpt5_result = gpt5_func()
        results["GPT-5"] = {
            'time': timer.get_time(),
            'result': gpt5_result,
            'success': True
        }
        print(f"Execution Time: {timer.get_time():.2f}s")
    except Exception as e:
        results["GPT-5"] = {'time': None, 'result': None, 'success': False, 'error': str(e)}
        print(f"Error: {e}")
    
    # Test GPT-4.1
    print(f"\n--- {azure_openai_gpt41_deployment_name} Results ---")
    try:
        with PerformanceTimer("GPT-4.1") as timer:
            gpt41_result = gpt41_func()
        results["GPT-4.1"] = {
            'time': timer.get_time(),
            'result': gpt41_result,
            'success': True
        }
        print(f"Execution Time: {timer.get_time():.2f}s")
    except Exception as e:
        results["GPT-4.1"] = {'time': None, 'result': None, 'success': False, 'error': str(e)}
        print(f"Error: {e}")
    
    # Comparison summary
    if results["GPT-5"]['success'] and results["GPT-4.1"]['success']:
        time_diff = results["GPT-5"]['time'] - results["GPT-4.1"]['time']
        faster_model = "GPT-5" if time_diff < 0 else "GPT-4.1"
        print("\n--- Performance Summary ---")
        print(f"{azure_openai_gpt5_deployment_name}: {results['GPT-5']['time']:.2f}s")
        print(f"{azure_openai_gpt41_deployment_name}: {results['GPT-4.1']['time']:.2f}s")
        print(f"Time Difference: {abs(time_diff):.2f}s ({faster_model} faster)")
    
    return results

# Print environment variables (excluding sensitive info)
print("=== Azure OpenAI Configuration ===")
print(f"Endpoint: {azure_openai_endpoint}")
print(f"API Version: {aoai_api_version}")
print(f"GPT-5 Deployment: {azure_openai_gpt5_deployment_name}")
print(f"GPT-4.1 Deployment: {azure_openai_gpt41_deployment_name}")
print(f"Embedding Deployment: {azure_openai_embedding_deployment}")
print(f"API Key set: {azure_openai_key is not None}")
print(f"Client initialized: {client is not None}")

=== Azure OpenAI Configuration ===
Endpoint: https://hyo-ai-foundry-pjt1-resource.openai.azure.com/
API Version: 2025-03-01-preview
GPT-5 Deployment: gpt-5-mini
GPT-4.1 Deployment: gpt-4.1
Embedding Deployment: text-embedding-ada-002
API Key set: True
Client initialized: True


## Basic Chat Test

In [16]:
# 1) Basic Chat Test with Performance Comparison
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me about Microsoft"}
]

def test_basic_chat_gpt5():
    response = client.chat.completions.create(
        model=azure_openai_gpt5_deployment_name,
        messages=messages,
        reasoning_effort = "low"  # low, medium, or high
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

def test_basic_chat_gpt41():
    response = client.chat.completions.create(
        model=azure_openai_gpt41_deployment_name,
        messages=messages
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

# Performance comparison
results = measure_and_compare("Basic Chat", test_basic_chat_gpt5, test_basic_chat_gpt41)


=== Basic Chat Performance Comparison ===

--- gpt-5-mini Results ---


Microsoft Corporation is an American multinational technology company founded in 1975 by Bill Gates and Paul Allen. Headquartered in Redmond, Washington, it’s one of the world’s largest and most influential tech companies, known for software, cloud computing, hardware, gaming, developer tools, and many enterprise services.

Key leadership and business focus
- CEO: Satya Nadella (since 2014). Under his leadership Microsoft shifted strongly toward cloud services, subscription models, and enterprise AI.
- Core strategy: cloud-first, AI-first — heavy investment in Microsoft Azure cloud platform and integrating AI capabilities across products.

Major products and services
- Windows: the company’s flagship desktop operating system (Windows 10, Windows 11).
- Microsoft 365 (formerly Office 365): subscription suite including Word, Excel, PowerPoint, Outlook, Teams, and OneDrive.
- Azure: cloud computing platform providing infrastructure-as-a-service (IaaS), platform-as-a-service (PaaS), and ma

## Function Calling Test

In [17]:
# 2) Function Calling Performance Comparison
def get_weather(city: str):
    """Mock weather function"""
    weather_data = {
        "Seoul": "Clear, 22°C",
        "Busan": "Cloudy, 19°C"
    }
    return weather_data.get(city, f"{city}: Weather information not available")

# function schema
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get weather for a city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name"}
            },
            "required": ["city"]
        }
    }
}]

# query include function calling
messages = [
    {"role": "user", "content": "Tell me the weather in Seoul"}
]

def test_function_calling_gpt5():
    response = client.chat.completions.create(
        model=azure_openai_gpt5_deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # function call check and execution
    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"function calling: {function_name}({arguments})")
        
        if function_name == "get_weather":
            result = get_weather(arguments["city"])
            print(f"function result: {result}")

            # generate final response with function result
            final_messages = messages.copy()
            final_messages.append(response.choices[0].message)
            final_messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
            
            final_response = client.chat.completions.create(
                model=azure_openai_gpt5_deployment_name,
                messages=final_messages,
                reasoning_effort = "low"  # low, medium, or high
            )

            print(f"final response: {final_response.choices[0].message.content}")
            return final_response.choices[0].message.content
    else:
        print(f"direct response: {response.choices[0].message.content}")
        return response.choices[0].message.content

def test_function_calling_gpt41():
    response = client.chat.completions.create(
        model=azure_openai_gpt41_deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    # function call check and execution
    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"function calling: {function_name}({arguments})")
        
        if function_name == "get_weather":
            result = get_weather(arguments["city"])
            print(f"function result: {result}")

            # generate final response with function result
            final_messages = messages.copy()
            final_messages.append(response.choices[0].message)
            final_messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
            
            final_response = client.chat.completions.create(
                model=azure_openai_gpt41_deployment_name,
                messages=final_messages
            )

            print(f"final response: {final_response.choices[0].message.content}")
            return final_response.choices[0].message.content
    else:
        print(f"direct response: {response.choices[0].message.content}")
        return response.choices[0].message.content

# Performance comparison
results = measure_and_compare("Function Calling", test_function_calling_gpt5, test_function_calling_gpt41)


=== Function Calling Performance Comparison ===

--- gpt-5-mini Results ---


function calling: get_weather({'city': 'Seoul'})
function result: Clear, 22°C
final response: Right now in Seoul it's clear and 22°C. Would you like a forecast for today or the next few days?
Execution Time: 2.59s

--- gpt-4.1 Results ---
function calling: get_weather({'city': 'Seoul'})
function result: Clear, 22°C
final response: The current weather in Seoul is clear with a temperature of 22°C. Let me know if you need a forecast or more details!
Execution Time: 0.97s

--- Performance Summary ---
gpt-5-mini: 2.59s
gpt-4.1: 0.97s
Time Difference: 1.62s (GPT-4.1 faster)


## Structured Output Test with Json Schema

In [18]:
# 3) Structured Output Performance Comparison (Chat Completions API - JSON Schema)
json_schema = {
    "name": "product_summary",
    "schema": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "category": {"type": "string"},
            "price_krw": {"type": "number"},
            "pros": {"type": "array", "items": {"type": "string"}},
            "cons": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["title", "category", "price_krw", "pros", "cons"], 
        "additionalProperties": False,
    },
    "strict": True,
}

messages = [
    {"role": "user", "content": "Pick a gaming laptop model randomly and generate a summary JSON. Include price in Korean Won with approximate values."}
]

def test_structured_output_gpt5():
    response = client.chat.completions.create(
        model=azure_openai_gpt5_deployment_name,
        messages=messages,
        response_format={"type": "json_schema", "json_schema": json_schema},
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

def test_structured_output_gpt41():
    response = client.chat.completions.create(
        model=azure_openai_gpt41_deployment_name,
        messages=messages,
        response_format={"type": "json_schema", "json_schema": json_schema},
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

# Performance comparison
results = measure_and_compare("Structured Output (JSON Schema)", test_structured_output_gpt5, test_structured_output_gpt41)


=== Structured Output (JSON Schema) Performance Comparison ===

--- gpt-5-mini Results ---


{"title":"Lenovo Legion 5 Pro (16) - Ryzen 7 / RTX 3070 Ti","category":"Gaming Laptop","price_krw":2490000,"pros":["Excellent CPU+GPU performance for modern AAA titles","16-inch 165Hz QHD display with good color and brightness","Robust cooling and sustained performance under load","Comfortable keyboard and solid build quality","Generally good value for performance compared to rivals"],"cons":["Relatively heavy and not very portable for frequent travel","Battery life is limited under gaming load","Webcam quality is mediocre","Can run warm and loud at max settings","Preinstalled software/bloatware on some units"]}
Execution Time: 4.61s

--- gpt-4.1 Results ---
{"title":"MSI Katana 15 B13VFK","category":"Gaming Laptop","price_krw":1800000,"pros":["Powerful Intel Core i7-13700H processor","NVIDIA GeForce RTX 4060 graphics","144Hz Full HD display for smooth gameplay","Good cooling system","Reasonable price for the specs"],"cons":["Average battery life","Plastic-heavy build quality","No Thun

## Responses API (stateful chat for 30days)

In [19]:
response = client.responses.create(   
  model=azure_openai_gpt5_deployment_name, 
  input="This is a test.",
)

print(response.model_dump_json(indent=2)) 

{
  "id": "resp_68be9a63df0c8195b9c221c01de2b7bd0041fbe4cc2f6317",
  "created_at": 1757321827.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [
    {
      "id": "rs_68be9a646ad08195997dbfdc528f44660041fbe4cc2f6317",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68be9a64e8748195bf8617116bb595560041fbe4cc2f6317",
      "content": [
        {
          "annotations": [],
          "text": "Received. I'm here — how can I help you test or what would you like to try?",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "max_output_t

## Responses API - List previous chats

In [20]:
response = client.responses.input_items.list("resp_68be60657b208194a9c1549d18e9223a0d97914cd52a20b6")

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_68be60657c2c8194a339915244faadfe0d97914cd52a20b6",
      "content": [
        {
          "text": "This is a test.",
          "type": "input_text"
        }
      ],
      "role": "user",
      "status": "completed",
      "type": "message"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_68be60657c2c8194a339915244faadfe0d97914cd52a20b6",
  "last_id": "msg_68be60657c2c8194a339915244faadfe0d97914cd52a20b6"
}


## Responses API - Upload PDF and analyze

In [21]:
# Responses API PDF Upload and Analysis Performance Comparison
# Upload a file with a purpose of "assistants"
try:
    file = client.files.create(
      file=open("AI_Brief_Aug2025.pdf", "rb"), # This assumes a .pdf file in the same directory as the executing script
      purpose="assistants"
    )
    print("File uploaded successfully:")
    print(file.model_dump_json(indent=2))
    file_id = file.id
    
    def test_responses_pdf_analysis_gpt5():
        response = client.responses.create(
            model=azure_openai_gpt5_deployment_name,
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "input_file",
                            "file_id": file_id
                        },
                        {
                            "type": "input_text",
                            "text": "Summarize this PDF in English in three sentences.",
                        },
                    ],
                },
            ]
        )
        print("--- GPT-5 Responses API PDF Analysis ---")
        print(response.model_dump_json(indent=2))
        return response.output_text if hasattr(response, 'output_text') else str(response.output)

    def test_responses_pdf_analysis_gpt41():
        response = client.responses.create(
            model=azure_openai_gpt41_deployment_name,
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "input_file",
                            "file_id": file_id
                        },
                        {
                            "type": "input_text",
                            "text": "Summarize this PDF in English in three sentences.",
                        },
                    ],
                },
            ]
        )
        print("--- GPT-4.1 Responses API PDF Analysis ---")
        print(response.model_dump_json(indent=2))
        return response.output_text if hasattr(response, 'output_text') else str(response.output)

    # Performance comparison
    results = measure_and_compare("Responses API PDF Analysis", test_responses_pdf_analysis_gpt5, test_responses_pdf_analysis_gpt41)
    
except FileNotFoundError:
    print("PDF file 'AI_Brief_Aug2025.pdf' not found. Please ensure the file exists in the current directory.")
except Exception as e:
    print(f"Error uploading or processing PDF: {e}")

File uploaded successfully:
{
  "id": "assistant-H8NQWkRDFeGHkoYnZuvBWE",
  "bytes": 1165436,
  "created_at": 1757321832,
  "filename": "AI_Brief_Aug2025.pdf",
  "object": "file",
  "purpose": "assistants",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}

=== Responses API PDF Analysis Performance Comparison ===

--- gpt-5-mini Results ---
--- GPT-5 Responses API PDF Analysis ---
{
  "id": "resp_68be9a690e488197be7f16d5563f49d40351cb2d174a3df2",
  "created_at": 1757321835.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [
    {
      "id": "rs_68be9a6d344881978764eef4512ed4ce0351cb2d174a3df2",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68be9a7161a48197904aa920143ab9d80351cb2d174a3df2",
      "content": [
        {
          "annotations": [],
     

## Responses API - Background Task (async)

In [22]:
response = client.responses.create(
    model = azure_openai_gpt5_deployment_name,
    input = "Write me a bedtime story (1min)",
    background = True
)

print(response.model_dump_json(indent=2))

{
  "id": "resp_68be9a7a19388190bdca4060ffa4bda50cf3668f435c231b",
  "created_at": 1757321850.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": true,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "prompt": null,
  "prompt_cache_key": null,
  "reasoning": {
    "effort": "medium",
    "generate_summary": null,
    "summary": null
  },
  "safety_identifier": null,
  "service_tier": "auto",
  "status": "queued",
  "text": {
    "format": {
      "type": "text"
    },
    "verbosity": null
  },
  "top_logprobs": null,
  "truncation": "disabled",
  "usage": null,
  "user": null,
  "content_filters": null,
  "store": true
}


In [23]:
from time import sleep

while response.status in {"queued", "in_progress"}:
    print(f"Current status: {response.status}")
    sleep(2)
    response = client.responses.retrieve(response.id)

print(f"Final status: {response.status}\nOutput:\n{response.output_text}")

Current status: queued


Current status: queued
Current status: queued
Final status: completed
Output:
A tiny star named Lila lived above a quiet village, tucked into the soft blue of the evening sky. Each night she would stretch her silver arms and hum very softly so the crickets could hear. Down below, a little fox curled up under a lantern, listening to Lila's hum as if it were a lullaby just for him. The moon leaned close and wrapped them both in a warm, pearly light. Trees sighed like sleepy giants and the river whispered secrets as it slid past smooth stones. Lila winked, sending one gentle sparkle that landed on the fox’s nose and made him smile in his sleep. All around, the world breathed out and settled, cozy as a blanket. The star whispered, "Rest now," and the night kept watch until morning. Sleep easy.


## Responses API - Function Calling Test

In [24]:
# Responses API Function Calling Performance Comparison
tools_responses = [  
    {  
        "type": "function",  
        "name": "get_weather",  
        "description": "Get the weather for a location",  
        "parameters": {  
            "type": "object",  
            "properties": {  
                "location": {"type": "string"},  
            },  
            "required": ["location"],  
        },  
    }  
]

def test_responses_function_calling_gpt5():
    response = client.responses.create(  
        model=azure_openai_gpt5_deployment_name, 
        tools=tools_responses,
        input=[{"role": "user", "content": "What's the weather in San Francisco?"}],  
    )  
    
    print("--- GPT-5 Responses API Function Calling ---")
    print(response.model_dump_json(indent=2))
    
    # To provide output to tools, add a response for each tool call to an array passed  
    # to the next response as `input`  
    input_data = []  
    for output in response.output:  
        if output.type == "function_call":  
            match output.name:  
                case "get_weather":  
                    input_data.append(  
                        {  
                            "type": "function_call_output",  
                            "call_id": output.call_id,  
                            "output": '{"temperature": "70 degrees"}',  
                        }  
                    )  
                case _:  
                    raise ValueError(f"Unknown function call: {output.name}")  
      
    second_response = client.responses.create(  
        model=azure_openai_gpt5_deployment_name,  
        previous_response_id=response.id,  
        input=input_data  
    )  
    
    print("--- GPT-5 Second Response ---")
    print(second_response.model_dump_json(indent=2))
    return second_response.output_text if hasattr(second_response, 'output_text') else str(second_response.output)

def test_responses_function_calling_gpt41():
    response = client.responses.create(  
        model=azure_openai_gpt41_deployment_name, 
        tools=tools_responses,
        input=[{"role": "user", "content": "What's the weather in San Francisco?"}],  
    )  
    
    print("--- GPT-4.1 Responses API Function Calling ---")
    print(response.model_dump_json(indent=2))
    
    # To provide output to tools, add a response for each tool call to an array passed  
    # to the next response as `input`  
    input_data = []  
    for output in response.output:  
        if output.type == "function_call":  
            match output.name:  
                case "get_weather":  
                    input_data.append(  
                        {  
                            "type": "function_call_output",  
                            "call_id": output.call_id,  
                            "output": '{"temperature": "70 degrees"}',  
                        }  
                    )  
                case _:  
                    raise ValueError(f"Unknown function call: {output.name}")  
      
    second_response = client.responses.create(  
        model=azure_openai_gpt41_deployment_name,  
        previous_response_id=response.id,  
        input=input_data  
    )  
    
    print("--- GPT-4.1 Second Response ---")
    print(second_response.model_dump_json(indent=2))
    return second_response.output_text if hasattr(second_response, 'output_text') else str(second_response.output)

# Performance comparison
results = measure_and_compare("Responses API Function Calling", test_responses_function_calling_gpt5, test_responses_function_calling_gpt41) 


=== Responses API Function Calling Performance Comparison ===

--- gpt-5-mini Results ---


--- GPT-5 Responses API Function Calling ---
{
  "id": "resp_68be9a813d9881938f447c2b66f36429024af8c1fa8f4650",
  "created_at": 1757321857.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [
    {
      "id": "rs_68be9a81c03481939c140e50b7465446024af8c1fa8f4650",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "arguments": "{\"location\":\"San Francisco, CA\"}",
      "call_id": "call_1y68o5Ev4wiQTZCMWMuGaOGI",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68be9a831f6081938ad4350ba836d3f3024af8c1fa8f4650",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "location": 

## Responses API - Structured Output Test with Json Schema

In [25]:
# 5) Structured Output (Responses API - JSON Schema)
# json_schema = {
#     "name": "product_summary",
#     "schema": {
#         "type": "object",
#         "properties": {
#             "title": {"type": "string"},
#             "category": {"type": "string"},
#             "price_krw": {"type": "number"},
#             "pros": {"type": "array", "items": {"type": "string"}},
#             "cons": {"type": "array", "items": {"type": "string"}},
#         },
#         "required": ["title", "category", "price_krw"],
#         "additionalProperties": False,
#     },
#     "strict": True,
# }

# inputs = [
#     {"role": "user", "content": "Pick a gaming laptop model randomly and generate a summary JSON. Include price in Korean Won with approximate values."}
# ]

# response = client.responses.create(
#     model=azure_openai_gpt5_deployment_name,
#     input=inputs,
#     response_format={"type": "json_schema", "json_schema": json_schema},
# )

# print(response.model_dump_json(indent=2)) 

print("Responses API doen't support response_format.")

Responses API doen't support response_format.


## Responses API - Streaming output Test

In [26]:
# 4) Streaming Output Performance Comparison
def test_streaming_gpt5():
    try:
        print("--- GPT-5 Streaming ---")
        stream = client.chat.completions.create(
            model=azure_openai_gpt5_deployment_name,
            messages=[{"role": "user", "content": "Tell me a five-line joke."}],
            stream=True,
        )
        
        content = ""
        for chunk in stream:
            if chunk.choices:
                delta = chunk.choices[0].delta.content
                if delta:
                    print(delta, end="", flush=True)
                    content += delta
                    time.sleep(0.05)
        print()
        return content
        
    except Exception as e:
        print(f"Streaming error: {e}")
        return None

def test_streaming_gpt41():
    try:
        print("--- GPT-4.1 Streaming ---")
        stream = client.chat.completions.create(
            model=azure_openai_gpt41_deployment_name,
            messages=[{"role": "user", "content": "Tell me a five-line joke."}],
            stream=True,
        )
        
        content = ""
        for chunk in stream:
            if chunk.choices:
                delta = chunk.choices[0].delta.content
                if delta:
                    print(delta, end="", flush=True)
                    content += delta
                    time.sleep(0.05)
        print()
        return content
        
    except Exception as e:
        print(f"Streaming error: {e}")
        return None

# Performance comparison
results = measure_and_compare("Streaming Output", test_streaming_gpt5, test_streaming_gpt41)


=== Streaming Output Performance Comparison ===

--- gpt-5-mini Results ---
--- GPT-5 Streaming ---


I told my smart speaker a joke.
It laughed for ten minutes.
I asked why it was so hyper.
It said, "Sorry, I was buffering."
Now we're both still processing.
Execution Time: 6.09s

--- gpt-4.1 Results ---
--- GPT-4.1 Streaming ---
Why did the scarecrow win an award?  
Because he was outstanding in his field!  
But then he got lonely,  
So he asked the corn for company—  
But it was too ear-resistible to leave!
Execution Time: 3.18s

--- Performance Summary ---
gpt-5-mini: 6.09s
gpt-4.1: 3.18s
Time Difference: 2.91s (GPT-4.1 faster)


## Responses API - Code Interpreter Test

In [27]:
# Responses API Code Interpreter Performance Comparison
instructions = "You are a personal math tutor. When asked a math question, write and run code using the python tool to answer the question."

tools_code_interpreter = [
    {
        "type": "code_interpreter",
        "container": {"type": "auto"}
    }
]

math_question = "I need to solve the equation 3x + 11 = 14. Can you help me?"

def test_responses_code_interpreter_gpt5():
    response = client.responses.create(
        model=azure_openai_gpt5_deployment_name,
        tools=tools_code_interpreter,
        instructions=instructions,
        input=math_question,
    )
    print("--- GPT-5 Responses API Code Interpreter ---")
    print(response.model_dump_json(indent=2))
    return response.output_text if hasattr(response, 'output_text') else str(response.output)

def test_responses_code_interpreter_gpt41():
    response = client.responses.create(
        model=azure_openai_gpt41_deployment_name,
        tools=tools_code_interpreter,
        instructions=instructions,
        input=math_question,
    )
    print("--- GPT-4.1 Responses API Code Interpreter ---")
    print(response.model_dump_json(indent=2))
    return response.output_text if hasattr(response, 'output_text') else str(response.output)

# Performance comparison
results = measure_and_compare("Responses API Code Interpreter", test_responses_code_interpreter_gpt5, test_responses_code_interpreter_gpt41) 


=== Responses API Code Interpreter Performance Comparison ===

--- gpt-5-mini Results ---


--- GPT-5 Responses API Code Interpreter ---
{
  "id": "resp_68be9a9099bc8190a1a98786b8d8b5c0069c4e70f2d46a9a",
  "created_at": 1757321872.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a personal math tutor. When asked a math question, write and run code using the python tool to answer the question.",
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [
    {
      "id": "rs_68be9a924acc8190819b051efcf8830e069c4e70f2d46a9a",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "ci_68be9a940a1081908d949618543be2b5069c4e70f2d46a9a",
      "code": "# Solve the equation 3x + 11 = 14\nfrom fractions import Fraction\nx = Fraction(14 - 11, 3)\nx, float(x), 3*x + 11\n",
      "container_id": "cntr_68be9a9152c08190bd0f857cb28c72110994bf39b83cf55d",
      "outputs": null,
      "status": "completed",
      "type": "code_interpreter_call"
    

## Responses API - Vision Support Test

In [28]:
# Responses API Vision Support Performance Comparison
# Note: Requires the deployed model to support vision and your API version to allow image input.
image_url = os.getenv("TEST_IMAGE_URL", "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Fronalpstock_big.jpg/640px-Fronalpstock_big.jpg")

inputs = [
    {"role": "user", "content": [
        {"type": "input_text", "text": "Summarize the main features of this image in three lines in English."},
        {"type": "input_image", "image_url": image_url},
    ]}
]

def test_responses_vision_gpt5():
    response = client.responses.create(
        model=azure_openai_gpt5_deployment_name, 
        input=inputs
    )
    print("--- GPT-5 Responses API Vision ---")
    print(response.model_dump_json(indent=2))
    return response.output_text if hasattr(response, 'output_text') else str(response.output)

def test_responses_vision_gpt41():
    response = client.responses.create(
        model=azure_openai_gpt41_deployment_name, 
        input=inputs
    )
    print("--- GPT-4.1 Responses API Vision ---")
    print(response.model_dump_json(indent=2))
    return response.output_text if hasattr(response, 'output_text') else str(response.output)

# Performance comparison
results = measure_and_compare("Responses API Vision Support", test_responses_vision_gpt5, test_responses_vision_gpt41)


=== Responses API Vision Support Performance Comparison ===

--- gpt-5-mini Results ---


--- GPT-5 Responses API Vision ---
{
  "id": "resp_68be9a9be9708190a59ab0c53edc30f50264ac3979a5c368",
  "created_at": 1757321884.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-mini",
  "object": "response",
  "output": [
    {
      "id": "rs_68be9a9f1fc08190b5abf0853617c8f90264ac3979a5c368",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68be9aa06d64819089f8cbe4bd5e1c7e0264ac3979a5c368",
      "content": [
        {
          "annotations": [],
          "text": "Panoramic mountain range with steep rocky cliffs and layered ridges.  \nLush green valleys and forests with scattered pastures and small buildings.  \nClear blue sky dotted with fluffy clouds and distant snow-capped peaks.",
          "type": "output_text",
          "logprobs": null
        }
      ],
      "role": "assistant",
      "status": "completed",
